In [6]:
from localtileserver import TileClient, get_leaflet_tile_layer
import leafmap.foliumap as leafmap

import pystac
import json
import os

os.environ["GTIFF_SRS_SOURCE"] = "GEOKEYS"

Load the JSON result listing generated by `cwltool`:

In [7]:
with open("/workspace/workshop/07_app_package/water-bodies/outputs/out.json") as f: 
    results = json.load(f)

Look for the `catalog.json` file:

In [8]:
for item in results["stac_catalog"]["listing"]:
    
    if item['basename'] == "catalog.json":
        catalog = pystac.read_file(item["path"])
        break
        

List the contents of the STAC Catalog

In [9]:
catalog.describe()

* <Catalog id=catalog>
  * <Item id=S2B_10TFK_20210713_0_L2A>
  * <Item id=S2A_10TFK_20220524_0_L2A>


In [10]:
m = leafmap.Map()

for item in catalog.get_all_items():
    
    m.add_raster(item.get_assets()["data"].get_absolute_href(), layer_name=item.id)

m

Warning 1: PROJ: proj_create_from_database: Open of /srv/conda/envs/env_visual/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /srv/conda/envs/env_visual/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /srv/conda/envs/env_visual/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /srv/conda/envs/env_visual/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /srv/conda/envs/env_visual/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /srv/conda/envs/env_visual/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /srv/conda/envs/env_visual/share/proj failed
